# Notebook 1: Homology matrix generation from genome sequences

This notebook is based on the 2020 tutorial published in Nature Protocols (PMID: 31863076). Notebook 1a-d is used to generate a draft GENRE for Gc strain FA1090. This notebook uses a model of Neisseria meningitidis MC58, Nmb_iTM560 (PMID: 22208880), as a scaffold. This notebook generates a homology matrices using the Neisseria meningitidis genome (AE002098.2) and Neisseria gonorrhoeae FA1090 genome (AE004969.1) from NCBI. 

Code and textual annotations from the 2020 protocol are preserved, and edited as appropriate for this application. 

This is the the first notebook in the tutorial to create homology matrix from genome sequences.There are four major steps in this notebook
1. Download the genome annotation (GenBank files) from NCBI, and generate fasta files (protein &nucleotide) from them
2. Perform BLASTp to find homologous proteins in strains of interest
3. Use best bidirectional hits to create gene presence/absence matrix
4. Supplementary for best practice: use BLASTn to check if we have missed any unannotated open reading frames and retain these genes in orthology matrix as well as guide future manual curation

In [1]:
#import packages needed
import pandas as pd
from glob import glob
from Bio import Entrez, SeqIO
import os

In [2]:
#Simplify reading/writing files
cwd=os.path.realpath(os.path.join(os.path.dirname(os.getcwd()),"..",".."))

In [3]:
os.path.realpath(os.path.join(os.path.dirname(os.getcwd()),"..",".."))

'C:\\Users\\Aimee\\Documents\\Metabolic_Modeling'

In [4]:
# Load the information on the strains we will be working with in this tutorial
StrainsOfInterest=pd.read_excel(cwd+"/Gc_GENRE_2022/Generate_Gc_Model/Strain_Information.xlsx")
StrainsOfInterest

,Strain,NCBI ID,Pathotype
0,Neisseria gonorrhoeae FA1090,AE004969.1,GC


In [5]:
#The Reference Genome is as described in Mendum 2011 for Nmb_iTM560 (PMID: 22208880)
referenceStrainID='AE002098.2'
targetStrainIDs=list(StrainsOfInterest['NCBI ID'])

## 1. Download genome annotations (GenBank files) to generate fasta files 

### Dowload genomes from NCBI
Download the genome annotations (GenBank files) from NCBI for strains of interest. 

In [6]:
# define a function to download the annotated genebank files from NCBI
def dl_genome(id, folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes'): # be sure get CORRECT ID
    files=glob('%s/*.gb'%folder)
    out_file = '%s/%s.gb'%(folder, id)

    if out_file in files:
        print (out_file, 'already downloaded')
        return
    else:
        print ('downloading %s from NCBI'%id)
        
    from Bio import Entrez
    Entrez.email = ""     #Insert email here for NCBI
    handle = Entrez.efetch(db="nucleotide", id=id, rettype="gb", retmode="text")
    fout = open(out_file,'w')
    fout.write(handle.read())
    fout.close()

In [7]:
# execute the above function, and download the GenBank files for the Gc strain
for strain in targetStrainIDs:
    dl_genome(strain, folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes')

downloading AE004969.1 from NCBI


### Examine the Downloaded Strains

In [8]:
# define a function to gather information of the downloaded strains from the GenBank files
def get_strain_info(folder='genomes'):
    files = glob('%s/*.gb'%folder)
    strain_info = []
    
    for file in files:
        handle = open(file)
        record = SeqIO.read(handle, "genbank")
        
        for f in record.features:
            if f.type=='source':
                info = {}
                info['file'] = file
                info['id'] = file.split('\\')[-1].split('.')[0]
                for q in f.qualifiers.keys():
                    info[q] = '|'.join(f.qualifiers[q])
                strain_info.append(info)
    return pd.DataFrame(strain_info)

In [9]:
# information on the downloaded strain
get_strain_info(folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes')

,file,id,organism,mol_type,strain,db_xref,note
0,C:\Users\Aimee\Documents\Metabolic_Modeling/Gc...,AE002098,Neisseria meningitidis MC58,genomic DNA,MC58,taxon:122586,serogroup: B
1,C:\Users\Aimee\Documents\Metabolic_Modeling/Gc...,AE004969,Neisseria gonorrhoeae FA 1090,genomic DNA,FA 1090,taxon:242231,NaN


### Generate FASTA files for both Protein and Nucleotide Pipelines
From the GenBank file, we can extract sequence and annotation information to generate fasta files for the protein and nucleotide analyses. The resulting fasta files will then be used in step 2 as input for BLAST 

In [10]:
# define a function to parse the Genbank file to generate fasta files for both protein and nucleotide sequences
def parse_genome(id, type='prot', in_folder='genomes', out_folder='prots', overwrite=1):

    in_file = '%s/%s.gb'%(in_folder, id)
    out_file='%s/%s.fa'%(out_folder, id)
    files =glob('%s/*.fa'%out_folder)
    
    if out_file in files and overwrite==0:
        print (out_file, 'already parsed')
        return
    else:
        print ('parsing %s'%id)
    
    handle = open(in_file)
    
    fout = open(out_file,'w')
    x = 0
    
    records = SeqIO.parse(handle, "genbank")
    for record in records:
        for f in record.features:
            if f.type=='CDS':
                seq=f.extract(record.seq)
                
                if type=='nucl':
                    seq=str(seq)
                else:
                    seq=str(seq.translate())
                    
                if 'locus_tag' in f.qualifiers.keys():
                    locus = f.qualifiers['locus_tag'][0]
                elif 'gene' in f.qualifiers.keys():
                    locus = f.qualifiers['gene'][0]
                else:
                    locus = 'gene_%i'%x
                    x+=1
                fout.write('>%s\n%s\n'%(locus, seq))
    fout.close()

In [11]:
# Generate fasta files for the Gc strain of interest
for strain in targetStrainIDs:
    parse_genome(strain, type='prot', in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes', out_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots')
    parse_genome(strain, type='nucl', in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes', out_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/nucl')


parsing AE004969.1
parsing AE004969.1


In [12]:
#Also generate fasta files for the reference strain
parse_genome(referenceStrainID, type='nucl', in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes', out_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/nucl')
parse_genome(referenceStrainID, type='prots', in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes', out_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots')

parsing AE002098.2
parsing AE002098.2


## 2. Perform BLAST to find homologous proteins in strains of interest

### Make BLAST DB for each of the target strains for both Protein and Nucleotide Pipelines

In this tutorial, we will run both BLASTp for proteins and BLSATn for nucleotides. BLASTp will be used as the main approach to identify homologous proteins in reference strain and other strains of interest, while BLASTn will be used as a supplementary method to check for any unannotated genes

In [13]:
# Define a function to make blast database for either protein of nucleotide
def make_blast_db(id,folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots',db_type='prot'):
    import os
    
    out_file ='%s/%s.fa.pin'%(folder, id)
    files =glob('%s/*.fa.pin'%folder)
    
    if out_file in files:
        print (id, 'already has a blast db')
        return
    if db_type=='nucl':
        ext='fna'
    else:
        ext='fa'

    cmd_line='makeblastdb -in %s/%s.%s -dbtype %s' %(folder, id, ext, db_type)
    
    print ('making blast db with following command line...')
    print (cmd_line)
    os.system(cmd_line)

In [14]:
# make protein sequence databases 
# Because we are performing bi-directional blast, we make databases from both reference strain and strains of interest
for strain in targetStrainIDs:
    make_blast_db(strain,folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots',db_type='prot')
make_blast_db(referenceStrainID,folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots',db_type='prot')

making blast db with following command line...
makeblastdb -in C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/prots/AE004969.1.fa -dbtype prot
making blast db with following command line...
makeblastdb -in C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/prots/AE002098.2.fa -dbtype prot


### Define functions to run protein BLAST and get sequence lengths
- BLASTp will be the main approach used here to identify homologous proteins between strains 
- Aside from sequence similarity, we also want to ensure the coverage of sequence mapping is sufficient. Therefore, we need to identiy the sequence length for each protein and compare it with the alignment length.

In [15]:
# define a function to run BLASTp
def run_blastp(seq,db,in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots', out_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/bbh', out=None,outfmt=6,evalue=0.001,threads=1):
    import os
    if out==None:
        out='%s/%s_vs_%s.txt'%(out_folder, seq, db)
        print(out)
    
    files =glob('%s/*.txt'%out_folder)
    if out in files:
        print (seq, 'already blasted')
        return
    
    print ('blasting %s vs %s'%(seq, db))
    
    db = '%s/%s.fa'%(in_folder, db)
    seq = '%s/%s.fa'%(in_folder, seq)
    cmd_line='blastp -db %s -query %s -out %s -evalue %s -outfmt %s -num_threads %i' \
    %(db, seq, out, evalue, outfmt, threads)
    
    print ('running blastp with following command line...')
    print (cmd_line)
    os.system(cmd_line)
    return out

In [16]:
# define a function to get sequence length 

def get_gene_lens(query, in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots'):

    file = '%s/%s.fa'%(in_folder, query)
    handle = open(file)
    records = SeqIO.parse(handle, "fasta")
    out = []
    
    for record in records:
        out.append({'gene':record.name, 'gene_length':len(record.seq)})
    
    out = pd.DataFrame(out)
    return out

## 3. Use Bi-Directional BLASTp Best Hits to create gene presence/absence matrix

### Obtain Bi-Directional BLASTp Best Hits

From the above BLASTp results, we can obtain Bi-Directional BLASTp Best Hits to identify homologous proteins. Note beside gene similarity score, the coverage of alignment is also used to filter mapping results. 

In [17]:
# define a function to get Bi-Directional BLASTp Best Hits
def get_bbh(query, subject, in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/bbh'):    
    
    #Utilize the defined protein BLAST function
    run_blastp(query, subject)
    run_blastp(subject, query)
    
    query_lengths = get_gene_lens(query, in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots')
    subject_lengths = get_gene_lens(subject, in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/prots')
    
    #Define the output file of this BLAST
    out_file = '%s/%s_vs_%s_parsed.csv'%(in_folder,query, subject)
    files=glob('%s/*_parsed.csv'%in_folder)
    
    #Combine the results of the protein BLAST into a dataframe
    print ('parsing BBHs for', query, subject)
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    bbh=pd.read_csv('%s/%s_vs_%s.txt'%(in_folder,query, subject), sep='\t', names=cols)
    bbh = pd.merge(bbh, query_lengths) 
    bbh['COV'] = bbh['alnLength']/bbh['gene_length']
    
    bbh2=pd.read_csv('%s/%s_vs_%s.txt'%(in_folder,subject, query), sep='\t', names=cols)
    bbh2 = pd.merge(bbh2, subject_lengths) 
    bbh2['COV'] = bbh2['alnLength']/bbh2['gene_length']
    out = pd.DataFrame()
    
    # Filter the genes based on coverage
    bbh = bbh[bbh.COV>=0.25]
    bbh2 = bbh2[bbh2.COV>=0.25]
    
    #Delineate the best hits from the BLAST
    for g in bbh.gene.unique():
        res = bbh[bbh.gene==g]
        if len(res)==0:
            continue
        best_hit = res.loc[res.PID.idxmax()]
        best_gene = best_hit.subject
        res2 = bbh2[bbh2.gene==best_gene]
        if len(res2)==0:
            continue
        best_hit2 = res2.loc[res2.PID.idxmax()]
        best_gene2 = best_hit2.subject
        if g==best_gene2:
            best_hit['BBH'] = '<=>'
        else:
            best_hit['BBH'] = '->'
        out=pd.concat([out, pd.DataFrame(best_hit).transpose()])
    
    #Save the final file to a designated CSV file    
    out.to_csv(out_file)

In [18]:
# Execute the BLAST for each target strain against the reference strain, save results to 'bbh' i.e. "bidirectional best
# hits" folder to create
# homology matrix

for strain in targetStrainIDs:
    get_bbh(referenceStrainID,strain, in_folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/bbh')

C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/bbh/AE002098.2_vs_AE004969.1.txt
blasting AE002098.2 vs AE004969.1
running blastp with following command line...
blastp -db C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/prots/AE004969.1.fa -query C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/prots/AE002098.2.fa -out C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/bbh/AE002098.2_vs_AE004969.1.txt -evalue 0.001 -outfmt 6 -num_threads 1
C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/bbh/AE004969.1_vs_AE002098.2.txt
blasting AE004969.1 vs AE002098.2
running blastp with following command line...
blastp -db C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/prots/AE002098.2.fa -query C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/prots/AE004969.1.fa -out C:\Users\Aimee\Documents\Metabolic_Modeling/

<ipython-input-17-c33a3115282b>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
C:\Users\Aimee\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
<ipython-input-17-c33a3115282b>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'


### Parse the BLAST Results into one Homology Matrix of the Reconstruction Genes

For the homology matrix, we only focus on genes that are present in the reference model

In [19]:
#Load all the BLAST files between the reference strain and target strains

blast_files=glob('%s/*_parsed.csv'%(cwd+'/Gc_GENRE_2022/Generate_Gc_Model/bbh'))

for blast in blast_files:
    bbh=pd.read_csv(blast)
    print (blast,bbh.shape) 

C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/bbh\AE002098.2_vs_AE004969.1_parsed.csv (1675, 16)


In [20]:
#Load the base reconstruction to designate the list of genes within the model
import cobra
model = cobra.io.load_json_model(cwd+'/Gc_GENRE_2022/Models/Nmb_iTM560.json')
listGeneIDs=[]
for gene in model.genes:
    listGeneIDs.append(gene.id)

In [21]:
#Create 2 matrices of N, rows where N is the number of model genes and M columns where M is the number of target strains
#One matrix will be populated with the PID results from the blasts and another with the mapping of gene locus tags

ortho_matrix=pd.DataFrame(index=listGeneIDs,columns=targetStrainIDs)
geneIDs_matrix=pd.DataFrame(index=listGeneIDs,columns=targetStrainIDs)

In [22]:
#Parse through each blast file and acquire pertinent information for each matrix for each of the base reconstruction genes
for blast in blast_files:
    bbh=pd.read_csv(blast)
    listIDs=[]
    listPID=[]
    for r,row in ortho_matrix.iterrows():
        try:
            currentOrtholog=bbh[bbh['gene']==r].reset_index()
            listIDs.append(currentOrtholog.iloc[0]['subject'])
            listPID.append(currentOrtholog.iloc[0]['PID'])
        except:
            listIDs.append('None')
            listPID.append(0)
    for col in ortho_matrix.columns:
        if col in blast:
            ortho_matrix[col]=listPID
            geneIDs_matrix[col]=listIDs

### Apply Similarity Threshold to Binarize  Homology Matrix to Presence/Absence Matrix
In this step, choose a threshold for the PID to determine if a gene is a absent/present in the strain of interest. We can then convert the homology matrix generated above into a binarized presence/absence matrix

In [23]:
# Genes with a greater than 80% PID are considered present in the target strain genome 
# and consequently less than 80% are considered absent from the target strain genome
for column in ortho_matrix:
    ortho_matrix.loc[ortho_matrix[column]<=80.0,column]=0
    ortho_matrix.loc[ortho_matrix[column]>80.0,column]=1

In [24]:
ortho_matrix

,AE004969.1
Sgene,0.0
NMB0319,1.0
NMB0182,1.0
NMB0453,1.0
NMB2039,0.0
...,...
NMB0930,0.0
NMB1315,1.0
NMB0544,1.0
NMB1823,1.0


## 4. Perform BLASTn to check unannotated open reading frames to guide manual curation 
At this juncture it may be useful to execute a supplementary nucleotide BLAST to check for unannotated genes, results here become candidates for manual curation. In this tutorial we retain unannotated genes that pass the threhsold for
similarity and contain no premature stop codons

In [25]:
#Define a function to generate FNA from the GBK files
def gbk2fasta(gbk_filename):
    faa_filename = '.'.join(gbk_filename.split('.')[:-1])+'.fna'
    input_handle  = open(gbk_filename, "r")
    output_handle = open(faa_filename, "w")

    for seq_record in SeqIO.parse(input_handle, "genbank") :
        print ("Converting GenBank record %s" % seq_record.id)
        output_handle.write(">%s %s\n%s\n" % (
               seq_record.id,
               seq_record.description,
               seq_record.seq))

    output_handle.close()
    input_handle.close()

In [26]:
#Define function to run the BLASTn
def run_blastn(seq, db,outfmt=6,evalue=0.001,threads=1):
    import os
    out = cwd+'/Gc_GENRE_2022/Generate_Gc_Model/nucl/'+seq+'_vs_'+db+'.txt'
    seq = cwd+'/Gc_GENRE_2022/Generate_Gc_Model/nucl/'+seq+'.fa'
    db = cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes/'+db+'.fna'
    
    cmd_line='blastn -db %s -query %s -out %s -evalue %s -outfmt %s -num_threads %i' \
    %(db, seq, out, evalue, outfmt, threads)
    
    print ('running blastn with following command line...')
    print (cmd_line)
    os.system(cmd_line)
    return out

In [27]:
# make nucleotide sequence databases 
for strain in targetStrainIDs:
    make_blast_db(strain,folder=cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes',db_type='nucl')

making blast db with following command line...
makeblastdb -in C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/genomes/AE004969.1.fna -dbtype nucl


In [28]:
# convert genbank files to fna files for strains of interest
for strain in targetStrainIDs:
    gbk2fasta(cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes/'+strain+'.gb')

Converting GenBank record AE004969.1


In [29]:
# perform uni-directional BLASTn hit
genome_blast_res=[]
for strain in targetStrainIDs:
    res = run_blastn(referenceStrainID,strain)
    genome_blast_res.append(res)

running blastn with following command line...
blastn -db C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/genomes/AE004969.1.fna -query C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/nucl/AE002098.2.fa -out C:\Users\Aimee\Documents\Metabolic_Modeling/Gc_GENRE_2022/Generate_Gc_Model/nucl/AE002098.2_vs_AE004969.1.txt -evalue 0.001 -outfmt 6 -num_threads 1


In [30]:
#define a function to parse through the nucleotide BLAST results and form one matrix of all the results
def parse_nucl_blast(infile):
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    data = pd.read_csv(infile, sep='\t', names=cols)
    data = data[(data['PID']>80) & (data['alnLength']>0.8*data['queryEnd'])]
    data2=data.groupby('gene').first()
    return data2.reset_index()


In [31]:
# parse the nucleotide blast matrix 
na_matrix=pd.DataFrame()
for file in genome_blast_res:
    genes =parse_nucl_blast(file)
    name ='.'.join(file.split('_')[-1].split('.')[:-1])
    na_matrix = na_matrix.append(genes[['gene','subject','PID']])
na_matrix = pd.pivot_table(na_matrix, index='gene', columns='subject',values='PID')

In [32]:
na_matrix.head()

subject,AE004969.1
gene,
NMB0001,97.561
NMB0002,94.458
NMB0003,91.911
NMB0004,97.764
NMB0005,98.023


### Examine unnannotated open reading frames
We compare the results from BLASTp and BLASTn and record any inconsistencies between the two matrices due to missing annotation. This result is then saved to guide future manual curation. 

In [33]:
# define a function to extract the sequence from fna file 
def extract_seq(g, contig, start, end):
    from Bio import SeqIO
    handle = open(g)
    records = SeqIO.parse(handle, "fasta")
    
    for record in records:
        if record.name==contig:
            if end>start:
                section = record[start:end]
            else:
                section = record[end-1:start+1].reverse_complement()
                
            seq = str(section.seq)
    return seq

In [34]:
#Define updated matrices that will include genes based on sequence evidence that were missing due to lack of annotation
ortho_matrix_w_unannotated = ortho_matrix.copy()
geneIDs_matrix_w_unannotated = geneIDs_matrix.copy()

In [35]:
#Define matrix of the BLASTn results for all the pertinent model genes
nonModelGenes=[]
for g in na_matrix.index:
    if g not in listGeneIDs:
        nonModelGenes.append(g)

na_model_genes=na_matrix.drop(nonModelGenes)

In [36]:
#For each strain in the ortho_matrix, identify genes that meet threshold of SEQ similarity, but missing from
#annotated ORFS. Additionally, look at the sequence to ensure that these cases do not have early stop codons indicating
#nonfunctional even if the NA seqs meet the threshold

pseudogenes = {}

for c in ortho_matrix.columns:
    
    orfs = ortho_matrix[c]
    genes = na_model_genes[c]
    # All the Model Genes that met the BLASTp Requirements
    orfs2 = orfs[orfs==1].index.tolist()
    # All the Model Genes based off of BLASTn similarity above threshold of 80
    genes2 = genes[genes>=80].index.tolist()
    # By Definition find the genes that pass sequence threshold but were NOT in annotated ORFs:
    unannotated = set(genes2) -set(orfs2)
    
    # Obtain sequences of this list to check for premature stop codons:
    data = cwd+'/Gc_GENRE_2022/Generate_Gc_Model/nucl/AE002098.2_vs_%s.txt'%c
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    data = pd.read_csv(data, sep='\t', names=cols)
    #
    pseudogenes[c] = {}
    unannotated_data = data[data['gene'].isin(list(unannotated))]
    for i in unannotated_data.index:
        gene = data.loc[i,'gene']
        contig = data.loc[i,'subject'] 
        start = data.loc[i,'subjectStart']
        end = data.loc[i,'subjectEnd']
        seq = extract_seq(cwd+'/Gc_GENRE_2022/Generate_Gc_Model/genomes/%s.fna'%c,contig, start, end)
        # check for early stop codons - these are likely nonfunctional and shouldn't be included
        if '*' in seq:
            print (seq)
            pseudogenes[c][gene]=seq
            # Remove the gene from list of unannotated genes
            unannotated-set([gene])
            
    
    print (c, unannotated)
    
    # For pertinent genes, retain those based off of nucleotide similarity within the orthology matrix and geneIDs matrix
    ortho_matrix_w_unannotated.loc[unannotated,c]=1
    for g in unannotated:
        geneIDs_matrix_w_unannotated.loc[g,c] = '%s_ortholog'%g
    

AE004969.1 {'NMB0460', 'NMB0825', 'NMB2039', 'NMB1069', 'NMB0209', 'NMB0716', 'NMB0884', 'NMB1476', 'NMB1151', 'NMB1395', 'NMB1668', 'NMB0881', 'NMB1268', 'NMB1968', 'NMB1703', 'NMB0466', 'NMB0930', 'NMB0389', 'NMB0671', 'NMB1723', 'NMB1190', 'NMB1540', 'NMB1189', 'NMB0892', 'NMB1730', 'NMB1429', 'NMB1057', 'NMB1304', 'NMB1064'}


In [37]:
#Save the Presence/Absence Matrix and geneIDs Matrix for future use
ortho_matrix_w_unannotated.to_csv(cwd+'/Gc_GENRE_2022/Generate_Gc_Model/Matrices/ortho_matrix.csv')
geneIDs_matrix_w_unannotated.to_csv(cwd+'/Gc_GENRE_2022/Generate_Gc_Model/Matrices/geneIDs_matrix.csv')